In [64]:
!pip install tensorflow

  Using cached tensorflow-2.13.0-cp310-cp310-macosx_10_15_x86_64.whl (216.2 MB)
  Using cached protobuf-4.23.4-cp37-abi3-macosx_10_9_universal2.whl (400 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.32.0-cp310-cp310-macosx_10_14_x86_64.whl (1.7 MB)
  Using cached libclang-16.0.0-py2.py3-none-macosx_10_9_x86_64.whl (26.7 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached grpcio-1.56.0.tar.gz (24.3 MB)
  Preparing metadata (setup.py) ... done
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl 

In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy.sparse import csr_matrix

# Data Prep

In [177]:
data=pd.read_csv("data_gpt.csv",names=['data'])

In [178]:
data.head()

,data
0,What is the process of opening a bank account?...
1,What are the requirements for obtaining a pers...
2,What is the legal age to open a bank account?;...
3,What are the penalties for late credit card pa...
4,What steps are involved in filing for bankrupt...


In [179]:
data.iloc[0][0].split(';')

['What is the process of opening a bank account?',
 "To open a bank account, you need to visit a branch, provide identification documents, fill out an application form, and deposit an initial amount. It's a straightforward process.",
 'Opening a bank account is a relatively simple process. You can go to a bank branch, bring your identification documents, complete the application form, and deposit an initial amount.',
 'Opening a bank account usually involves visiting a branch, providing necessary documents, filling out an application form, and depositing an initial amount. It may vary slightly depending on the bank.']

In [180]:
X_train=[]
X_test = []
y_train=[]
y_test=[]
for i in range(int(0.9*data.shape[0])):
    row = data.iloc[i][0].split(';')
    X_train.append(row[1])
    y_train.append("highly assertive")
    X_train.append(row[2])
    y_train.append("moderately assertive")
    X_train.append(row[3])
    y_train.append("not assertive")
for i in range(int(0.9*data.shape[0]),data.shape[0]):
    row = data.iloc[i][0].split(';')
    X_test.append(row[1])
    y_test.append("highly assertive")
    X_test.append(row[2])
    y_test.append("moderately assertive")
    X_test.append(row[3])
    y_test.append("not assertive")

In [181]:
len(X_train),len(X_test),len(y_train),len(y_test)

(75, 9, 75, 9)

# Data Vectorizer

In [208]:
vectorizer = TfidfVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [209]:
# vectorizer = TfidfVectorizer(ngram_range=(1,2))
# X_train_transformed = vectorizer.fit_transform(X_train)
# X_test_transformed = vectorizer.transform(X_test)

In [210]:
X_train_transformed.shape

(75, 760)

# SVC classifier

In [211]:

svc_classifier = LinearSVC()
svc_classifier.fit(X_train_transformed, y_train)


LinearSVC()

In [212]:
y_pred = svc_classifier.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print("testing Accuracy svc:", accuracy)

testing Accuracy svc: 0.4444444444444444


In [213]:
confusion_matrix(y_pred,y_test)

array([[1, 1, 1],
       [2, 1, 0],
       [0, 1, 2]])

In [214]:
y_pred = svc_classifier.predict(X_train_transformed)
accuracy = accuracy_score(y_train, y_pred)
print("traing Accuracy svc:", accuracy)

traing Accuracy svc: 1.0


In [215]:
confusion_matrix(y_pred,y_train)

array([[25,  0,  0],
       [ 0, 25,  0],
       [ 0,  0, 25]])

# Logistic classifier

In [216]:
linear_classifier = LogisticRegression()
linear_classifier.fit(X_train_transformed, y_train)

LogisticRegression()

In [217]:
y_pred = linear_classifier.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy Linear:", accuracy)

Testing Accuracy Linear: 0.5555555555555556


In [218]:
confusion_matrix(y_pred,y_test)

array([[1, 0, 1],
       [2, 2, 0],
       [0, 1, 2]])

In [219]:
y_pred = linear_classifier.predict(X_train_transformed)
accuracy = accuracy_score(y_train, y_pred)
print("traing Accuracy Linear:", accuracy)

traing Accuracy Linear: 1.0


In [220]:
confusion_matrix(y_pred,y_train)

array([[25,  0,  0],
       [ 0, 25,  0],
       [ 0,  0, 25]])

# Neural Network

In [295]:
y_train_nn = np.zeros((len(y_train),3))
for i in range(len(y_train)):
    if y_train[i]=='highly assertive':
        y_train_nn[i][0]=1
    elif y_train[i]=='moderately assertive':
        y_train_nn[i][1]=1
    elif y_train[i]=='not assertive':
        y_train_nn[i][2]=1
y_test_nn = np.zeros((len(y_test),3))
for i in range(len(y_test)):
    if y_test[i]=='highly assertive':
        y_test_nn[i][0]=1
    elif y_test[i]=='moderately assertive':
        y_test_nn[i][1]=1
    elif y_test[i]=='not assertive':
        y_test_nn[i][2]=1

In [296]:
model = Sequential()
model.add(Dense(512, input_shape=(X_train_transformed.shape[1],), activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [297]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [298]:
X_train_transformed=csr_matrix(X_train_transformed).todense()
X_test_transformed=csr_matrix(X_test_transformed).todense()

In [299]:
model.fit(X_train_transformed,y_train_nn,epochs=10,batch_size=32)

Epoch 1/10
3/3 [==============================] - 0s 6ms/step - loss: 1.1089 - accuracy: 0.2000
Epoch 2/10
3/3 [==============================] - 0s 7ms/step - loss: 1.0661 - accuracy: 0.7733
Epoch 3/10
3/3 [==============================] - 0s 11ms/step - loss: 1.0341 - accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 0s 12ms/step - loss: 0.9963 - accuracy: 0.9867
Epoch 5/10
3/3 [==============================] - 0s 10ms/step - loss: 0.9429 - accuracy: 0.9733
Epoch 6/10
3/3 [==============================] - 0s 10ms/step - loss: 0.8716 - accuracy: 0.9867
Epoch 7/10
3/3 [==============================] - 0s 10ms/step - loss: 0.7682 - accuracy: 1.0000
Epoch 8/10
3/3 [==============================] - 0s 11ms/step - loss: 0.6368 - accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 0s 9ms/step - loss: 0.4791 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 6ms/step - loss: 0.3268 - accuracy: 0.9867


In [300]:
y_pred=model.predict(X_test_transformed).argmax(axis=1)
y_actual=y_test_nn.argmax(axis=1)

1/1 [==============================] - 0s 41ms/step


In [301]:
(y_pred==y_actual).sum()/y_actual.shape[0]

0.5555555555555556

In [302]:
confusion_matrix(y_pred,y_actual)

array([[1, 0, 0],
       [0, 2, 1],
       [2, 1, 2]])

In [303]:
y_pred=model.predict(X_train_transformed).argmax(axis=1)
y_actual=y_train_nn.argmax(axis=1)

3/3 [==============================] - 0s 3ms/step


In [304]:
(y_pred==y_actual).sum()/y_actual.shape[0]

0.9866666666666667

In [305]:
confusion_matrix(y_pred,y_actual)

array([[24,  0,  0],
       [ 0, 25,  0],
       [ 1,  0, 25]])